In [1]:
import requests, zipfile, io
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from PIL import Image
from io import BytesIO
import random
import tensorflow as tf
import time
import tensorflow.keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.losses import categorical_crossentropy
import requests



In [2]:
#load input
#600x1800x1800
#CHANGE TO TMC
#x = np.load('xAll0.npy')
x = np.load('x75Convex.npy')
x1 = np.load('xAll1.npy')
#x = np.random.normal(size = (600, 1800,1800))
#x1 = np.random.normal(size = (100, 1800, 1800))

In [3]:
x = x.reshape(-1,1800,1800, 1)
x1 = x1.reshape(-1,1800,1800, 1)
print(x.shape)

(75, 1800, 1800, 1)


In [4]:
N_VAL = 150
N_TRAIN = 150
train_examples = x[:N_TRAIN,:,:,:]
val_examples = x1[:N_VAL, :, :, :]

In [5]:
#y = np.load('y0.npy')
y = np.load('y75Convex.npy')
y1 = np.load('y1.npy')
#y = np.random.choice([0,1], size = (600, 8))
#y1 = np.random.choice([0,1], size = (N_VAL,8))
print(y.shape)

(75, 8)


In [6]:
#generate random NxD array (N images/vecs - 600, D diagnoses - 8)
#train input on this

#train_labels = np.random.choice([0,1], (N,D))
#print(train_labels.shape)
val_labels = y1[:N_VAL,:]#diag0[500:, :]
train_labels = y[:N_TRAIN,:] #diag0[:500, :]


In [7]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_examples, train_labels))
val_dataset = tf.data.Dataset.from_tensor_slices((val_examples, val_labels))
print(len(train_dataset), train_dataset)

BATCH_SIZE = 32
train_dataset = train_dataset.batch(BATCH_SIZE)
val_dataset = val_dataset.batch(BATCH_SIZE)
print(len(train_dataset), train_dataset)

75 <TensorSliceDataset shapes: ((1800, 1800, 1), (8,)), types: (tf.float16, tf.float64)>
3 <BatchDataset shapes: ((None, 1800, 1800, 1), (None, 8)), types: (tf.float16, tf.float64)>


In [8]:
#AUTOTUNE = tf.data.experimental.AUTOTUNE
#train_dataset = train_dataset.prefetch(buffer_size = AUTOTUNE)
#val_dataset = val_dataset.prefetch(buffer_size = AUTOTUNE)
#print(len(train_dataset), train_dataset)

In [ ]:
models = [(2, 1200, 600),(3, 1200, 600), (2, 1000, 800), (3, 1000, 800)]
numReg = 4
numLr = 3
numEpochs = 15
#models = [(3,1200, 8),...] #filter, size of FC1, size FC2

#6 conv, 3 dense, use layernorms pls
#try filter size 3 and 2 and 4
#try different size dense layers
input_tensor = tf.keras.Input(shape=(1800,1800,1) )
best_acc = 0
best_model = None


###init

reg = 7.487 #(np.random.rand()*5) + 3
lr = 5.102 #(np.random.rand()*4) + 2

#3.3701311397595526_5.2590542363191535_(2, 1000, 800).csv
r = 10**(-7.487)
learningRate = 10**(-5.102)
filter_size, FC1_size, FC2_size = (2,1200,600)
input_tensor = tf.keras.Input(shape=(1800,1800,1) )

###set up model
##CNNs & layer norms
#strides - default(1)
c1 = Conv2D(3, kernel_size=(filter_size, filter_size), activation='relu', 
            input_shape=(1800,1800,1), kernel_regularizer = tf.keras.regularizers.L2(r))(input_tensor) 
l1 = tf.keras.layers.LayerNormalization()(c1)
mp1 = tf.keras.layers.MaxPool2D()(l1)

c2 = Conv2D(3, kernel_size=(filter_size, filter_size), activation='relu', 
            kernel_regularizer = tf.keras.regularizers.L2(r))(mp1)
l2 = tf.keras.layers.LayerNormalization()(c2)
mp2 = tf.keras.layers.MaxPool2D()(l2)

c3 = Conv2D(3, kernel_size=(filter_size, filter_size), activation='relu', 
            kernel_regularizer = tf.keras.regularizers.L2(r))(mp2)
l3 = tf.keras.layers.LayerNormalization()(c3)
mp3 = tf.keras.layers.MaxPool2D()(l3)

c4 = Conv2D(3, kernel_size=(filter_size, filter_size), activation='relu', 
kernel_regularizer = tf.keras.regularizers.L2(r))(mp3)
l4 = tf.keras.layers.LayerNormalization()(c4)
mp4 = tf.keras.layers.MaxPool2D()(l4)

c5 = Conv2D(3, kernel_size=(filter_size, filter_size), activation='relu', 
kernel_regularizer = tf.keras.regularizers.L2(r))(mp4)
l5 = tf.keras.layers.LayerNormalization()(c5)
mp5 = tf.keras.layers.MaxPool2D()(l5)

c6 = Conv2D(3, kernel_size=(filter_size, filter_size), activation='relu', 
kernel_regularizer = tf.keras.regularizers.L2(r))(mp5)
l6 = tf.keras.layers.LayerNormalization()(c6)
mp6 = tf.keras.layers.MaxPool2D()(l6)

##FCs
head_out = Flatten()(mp6)

FC1 = Dense(FC1_size, activation = 'relu', kernel_regularizer = tf.keras.regularizers.L2(r))(head_out)
l7 = tf.keras.layers.LayerNormalization()(FC1)
FC2 = Dense(FC2_size, activation = 'relu', kernel_regularizer = tf.keras.regularizers.L2(r))(l7)
predictions = Dense(8, activation = 'sigmoid', kernel_regularizer = tf.keras.regularizers.L2(r)) (FC2)

#connect model
model = Model(inputs=input_tensor,outputs=predictions)
#run
model.compile(loss= categorical_crossentropy,
              optimizer=Adam(learning_rate=learningRate),
              metrics=['accuracy'])
#runs
model.summary()
history = model.fit(train_dataset, validation_data = val_dataset, epochs = numEpochs)

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 1800, 1800, 1)]   0         
_________________________________________________________________
conv2d (Conv2D)              (None, 1799, 1799, 3)     15        
_________________________________________________________________
layer_normalization (LayerNo (None, 1799, 1799, 3)     6         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 899, 899, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 898, 898, 3)       39        
_________________________________________________________________
layer_normalization_1 (Layer (None, 898, 898, 3)       6         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 449, 449, 3)      

In [ ]:
#save
fname = 'data_selection/conv_tmc_convex/'+str(reg)+'_'+str(lr)+'_(2,1200,600)_' + '15_epochs'
hist = pd.DataFrame(history.history)
hist.to_csv(fname+'.csv')

model.save('data_selection/conv_tmc_convex/')

In [ ]:
#test
N_TEST = 150
test_examples = x1[-N_TEST:, :, :, :]
test_dataset = tf.data.Dataset.from_tensor_slices((test_examples))


In [ ]:
print(test_examples.shape)
print(len(test_dataset), test_dataset)
#Batch = 32 ? rn its 64
test_dataset = test_dataset.batch(BATCH_SIZE)
print(len(test_dataset), test_dataset)

In [ ]:
predictions = model.predict(test_dataset)
print(predictions.shape)
print(predictions[:5])

In [ ]:
#save test
fname = 'data_selection/conv_tmc_convex/preds_'+str(reg)+'_'+str(lr)+'_(2,1200,600)_' + '15_epochs'
pred = pd.DataFrame(predictions)
pred.to_csv(fname+'.csv')

In [ ]:
history.history

In [ ]:
#lded = pd.read_csv(fname+'.csv')

In [ ]:
#lded

In [ ]:
# Fit data to model
#history = model.fit(train_dataset, validation_data = val_dataset, epochs = 5)

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(15,15))
axs[0].plot(history.history['loss'])
axs[0].plot(history.history['val_loss'])
axs[0].title.set_text('Training Loss vs Validation Loss')
axs[0].set_xlabel('Epochs')
axs[0].set_ylabel('Loss')
axs[0].legend(['Train','Val'])
axs[1].plot(history.history['accuracy'])
axs[1].plot(history.history['val_accuracy'])
axs[1].title.set_text('Training Accuracy vs Validation Accuracy')
axs[1].set_xlabel('Epochs')
axs[1].set_ylabel('Accuracy')
axs[1].legend(['Train', 'Val'])